In [43]:
import keras as k
from keras.applications.vgg16 import VGG16
import pandas as pd
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import svm
from scipy.stats import mode
from sklearn.multiclass import OneVsOneClassifier

In [44]:
# Extract and create the data set
data = pd.read_csv("data/micrograph.csv")

In [45]:
data_spheroidite = data.loc[data['primary_microconstituent'] == 'spheroidite'].copy()
data_network = data.loc[data['primary_microconstituent'] == 'network'].copy()
data_pearlite = data.loc[data['primary_microconstituent'] == 'pearlite'].copy()


img = image.img_to_array(image.load_img("data/micrograph/"+data_spheroidite['path'].iloc[0]))
img = img[0:484,:,:]
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

In [46]:
model = VGG16(weights = 'imagenet', include_top=False)

In [47]:
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=[model.get_layer(index=3).output, 
                                          model.get_layer(index=6).output,
                                          model.get_layer(index=10).output,
                                          model.get_layer(index=14).output,
                                          model.get_layer(index=18).output])



In [6]:
def extract_features (data):
    data['c12'] = [np.empty(1) for i in range(data.shape[0])]
    data['c22'] = [np.empty(1) for i in range(data.shape[0])]
    data['c33'] = [np.empty(1) for i in range(data.shape[0])]
    data['c43'] = [np.empty(1) for i in range(data.shape[0])]
    data['c53'] = [np.empty(1) for i in range(data.shape[0])]
    
    for index, row in data.iterrows():
        img = image.img_to_array(image.load_img("data/micrograph/"+row['path']))
        img = img[0:484,:,:]
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        c12, c22, c33, c43, c53 = intermediate_layer_model.predict(img)
        data.at[index,'c12'] = c12.mean(axis=(0,1,2,))
        data.at[index,'c22'] = c22.mean(axis=(0,1,2,))
        data.at[index,'c33'] = c33.mean(axis=(0,1,2,))
        data.at[index,'c43'] = c43.mean(axis=(0,1,2,))
        data.at[index,'c53'] = c53.mean(axis=(0,1,2,))

In [7]:
%timeit
extract_features (data_spheroidite)
%timeit
extract_features (data_network)
%timeit
extract_features (data_pearlite)

In [8]:
#train data

train_data_spher_netwk = pd.DataFrame ({'c12': np.append(data_spheroidite[0:100]['c12'].values, data_network[0:100]['c12'].values),
                                   'c22': np.append(data_spheroidite[0:100]['c22'].values, data_network[0:100]['c22'].values),
                                   'c33': np.append(data_spheroidite[0:100]['c33'].values, data_network[0:100]['c33'].values),
                                   'c43': np.append(data_spheroidite[0:100]['c43'].values, data_network[0:100]['c43'].values),
                                   'c53': np.append(data_spheroidite[0:100]['c53'].values, data_network[0:100]['c53'].values),
                                   'type': np.append(np.ones(100), 2*np.ones(100))}).sample(frac=1, random_state=0)

train_data_netwk_pearl = pd.DataFrame ({'c12': np.append(data_network[0:100]['c12'].values, data_pearlite[0:100]['c12'].values),
                                   'c22': np.append(data_network[0:100]['c22'].values, data_pearlite[0:100]['c22'].values),
                                   'c33': np.append(data_network[0:100]['c33'].values, data_pearlite[0:100]['c33'].values),
                                   'c43': np.append(data_network[0:100]['c43'].values, data_pearlite[0:100]['c43'].values),
                                   'c53': np.append(data_network[0:100]['c53'].values, data_pearlite[0:100]['c53'].values),
                                   'type': np.append(2*np.ones(100), 3*np.ones(100))}).sample(frac=1, random_state=0)

train_data_pearl_spher = pd.DataFrame ({'c12': np.append(data_pearlite[0:100]['c12'].values, data_spheroidite[0:100]['c12'].values),
                                   'c22': np.append(data_pearlite[0:100]['c22'].values, data_spheroidite[0:100]['c22'].values),
                                   'c33': np.append(data_pearlite[0:100]['c33'].values, data_spheroidite[0:100]['c33'].values),
                                   'c43': np.append(data_pearlite[0:100]['c43'].values, data_spheroidite[0:100]['c43'].values),
                                   'c53': np.append(data_pearlite[0:100]['c53'].values, data_spheroidite[0:100]['c53'].values),
                                   'type' : np.append(3*np.ones(100), np.ones(100))}).sample(frac=1, random_state=0)


In [9]:
# test_data

test_data_spher_netwk = pd.DataFrame ({'c12': np.append(data_spheroidite[100:]['c12'].values, data_network[100:]['c12'].values),
                                   'c22': np.append(data_spheroidite[100:]['c22'].values, data_network[100:]['c22'].values),
                                   'c33': np.append(data_spheroidite[100:]['c33'].values, data_network[100:]['c33'].values),
                                   'c43': np.append(data_spheroidite[100:]['c43'].values, data_network[100:]['c43'].values),
                                   'c53': np.append(data_spheroidite[100:]['c53'].values, data_network[100:]['c53'].values),
                                   'type' : np.append(np.ones(data_spheroidite[100:].shape[0]), 2*np.ones(data_network[100:].shape[0]))})

test_data_netwk_pearl = pd.DataFrame ({'c12': np.append(data_network[100:]['c12'].values, data_pearlite[100:]['c12'].values),
                                   'c22': np.append(data_network[100:]['c22'].values, data_pearlite[100:]['c22'].values),
                                   'c33': np.append(data_network[100:]['c33'].values, data_pearlite[100:]['c33'].values),
                                   'c43': np.append(data_network[100:]['c43'].values, data_pearlite[100:]['c43'].values),
                                   'c53': np.append(data_network[100:]['c53'].values, data_pearlite[100:]['c53'].values),
                                   'type' : np.append(2*np.ones(data_network[100:].shape[0]), 3*np.ones(data_pearlite[100:].shape[0]))})

test_data_pearl_spher = pd.DataFrame ({'c12': np.append(data_pearlite[100:]['c12'].values, data_spheroidite[100:]['c12'].values),
                                   'c22': np.append(data_pearlite[100:]['c22'].values, data_spheroidite[100:]['c22'].values),
                                   'c33': np.append(data_pearlite[100:]['c33'].values, data_spheroidite[100:]['c33'].values),
                                   'c43': np.append(data_pearlite[100:]['c43'].values, data_spheroidite[100:]['c43'].values),
                                   'c53': np.append(data_pearlite[100:]['c53'].values, data_spheroidite[100:]['c53'].values),
                                   'type' : np.append(3*np.ones(data_pearlite[100:].shape[0]), np.ones(data_spheroidite[100:].shape[0]))})


In [11]:
#spheroidite and network

print ("Cross validation error for spheroidite and network classifier: \n")

# c12
sp_nt_clf12 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(sp_nt_clf12, list(train_data_spher_netwk['c12'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 error : ", 1.0 - scores['test_score'].mean())

# c22
sp_nt_clf22 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(sp_nt_clf22, list(train_data_spher_netwk['c22'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 error : ", 1.0 - scores['test_score'].mean())

# c33
sp_nt_clf33 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(sp_nt_clf33, list(train_data_spher_netwk['c33'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 error : ", 1.0 - scores['test_score'].mean())

# c43
sp_nt_clf43 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(sp_nt_clf43, list(train_data_spher_netwk['c43'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 error : ", 1.0 - scores['test_score'].mean())

# c53
sp_nt_clf53 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(sp_nt_clf53, list(train_data_spher_netwk['c53'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 error : ", 1.0 - scores['test_score'].mean())

Cross validation error for spheroidite and network classifier: 

c12 error :  0.5
c22 error :  0.5
c33 error :  0.495
c43 error :  0.49
c53 error :  0.04000000000000026


In [12]:
# c12 error :  0.5
# c22 error :  0.5
# c33 error :  0.495
# c43 error :  0.49
# c53 error :  0.04000000000000026

In [13]:
#network and pearlite

print ("Cross validation error for network and pearlite classifier: \n")

# c12
nt_pl_clf12 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(nt_pl_clf12, list(train_data_netwk_pearl['c12'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 error : ", 1.0 - scores['test_score'].mean())

# c22
nt_pl_clf22 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(nt_pl_clf22, list(train_data_netwk_pearl['c22'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 error : ", 1.0 - scores['test_score'].mean())

# c33
nt_pl_clf33 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(nt_pl_clf33, list(train_data_netwk_pearl['c33'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 error : ", 1.0 - scores['test_score'].mean())

# c43
nt_pl_clf43 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(nt_pl_clf43, list(train_data_netwk_pearl['c43'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 error : ", 1.0 - scores['test_score'].mean())

# c53
nt_pl_clf53 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(nt_pl_clf53, list(train_data_netwk_pearl['c53'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 error : ", 1.0 - scores['test_score'].mean())

Cross validation error for network and pearlite classifier: 

c12 error :  0.49
c22 error :  0.48
c33 error :  0.5
c43 error :  0.495
c53 error :  0.025000000000000133


In [14]:
# c12 error :  0.49
# c22 error :  0.48
# c33 error :  0.5
# c43 error :  0.495
# c53 error :  0.025000000000000133

In [15]:
#pearlite and spheroidite

print ("Cross validation error for pearlite and spheroidite classifier: \n")


# c12
pl_sp_clf12 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(pl_sp_clf12, list(train_data_pearl_spher['c12'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 error : ", 1.0 - scores['test_score'].mean())

# c22
pl_sp_clf22 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(pl_sp_clf22, list(train_data_pearl_spher['c22'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 error : ", 1.0 - scores['test_score'].mean())

# c33
pl_sp_clf33 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(pl_sp_clf33, list(train_data_pearl_spher['c33'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 error : ", 1.0 - scores['test_score'].mean())

# c43
pl_sp_clf43 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(pl_sp_clf43, list(train_data_pearl_spher['c43'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 error : ", 1.0 - scores['test_score'].mean())

# c53
pl_sp_clf53 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
scores = cross_validate(pl_sp_clf53, list(train_data_pearl_spher['c53'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 error : ", 1.0 - scores['test_score'].mean())

Cross validation error for pearlite and spheroidite classifier: 

c12 error :  0.5
c22 error :  0.4850000000000001
c33 error :  0.4850000000000001
c43 error :  0.495
c53 error :  0.050000000000000044


In [16]:
# c12 error :  0.5
# c22 error :  0.4850000000000001
# c33 error :  0.4850000000000001
# c43 error :  0.495
# c53 error :  0.050000000000000044

In [17]:
sp_nt_clf53 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
sp_nt_clf53.fit(list(train_data_spher_netwk['c53'].values), list(train_data_spher_netwk['type'].values))
print("SP_NT classifier test error : ", 1.0 - sp_nt_clf53.score(list(test_data_spher_netwk['c53'].values), list(test_data_spher_netwk['type'].values)))

SP_NT classifier test error :  0.033678756476683946


In [18]:
# SP_NT classifier test error :  0.033678756476683946

In [19]:
nt_pl_clf53 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
nt_pl_clf53.fit(list(train_data_netwk_pearl['c53'].values), list(train_data_netwk_pearl['type'].values))
print("NT_PL classifier test error : ", 1.0 - nt_pl_clf53.score(list(test_data_netwk_pearl['c53'].values), list(test_data_netwk_pearl['type'].values)))

NT_PL classifier test error :  0.07352941176470584


In [20]:
# NT_PL classifier test error :  0.07352941176470584

In [21]:
pl_sp_clf53 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
pl_sp_clf53.fit(list(train_data_pearl_spher['c53'].values), list(train_data_pearl_spher['type'].values))
print("PL_SP classifier test error  : ", 1.0 - pl_sp_clf53.score(list(test_data_pearl_spher['c53'].values), list(test_data_pearl_spher['type'].values)))

PL_SP classifier test error  :  0.0


In [22]:
# PL_SP classifier test error  :  0.0

In [23]:
train_data = pd.DataFrame ({'c12': np.append(np.append(data_spheroidite[0:100]['c12'].values, data_network[0:100]['c12'].values), data_pearlite[0:100]['c12'].values),
                                   'c22': np.append(np.append(data_spheroidite[0:100]['c22'].values, data_network[0:100]['c22'].values), data_pearlite[0:100]['c22'].values),
                                   'c33': np.append(np.append(data_spheroidite[0:100]['c33'].values, data_network[0:100]['c33'].values), data_pearlite[0:100]['c33'].values),
                                   'c43': np.append(np.append(data_spheroidite[0:100]['c43'].values, data_network[0:100]['c43'].values), data_pearlite[0:100]['c43'].values),
                                   'c53': np.append(np.append(data_spheroidite[0:100]['c53'].values, data_network[0:100]['c53'].values), data_pearlite[0:100]['c53'].values),
                                   'type': np.append(np.append(np.ones(100), 2*np.ones(100)), 3*np.ones(100))}).sample(frac=1, random_state=0)



test_data = pd.DataFrame ({'c12': np.append(np.append(data_spheroidite[100:]['c12'].values, data_network[100:]['c12'].values), data_pearlite[100:]['c12'].values),
                                   'c22': np.append(np.append(data_spheroidite[100:]['c22'].values, data_network[100:]['c22'].values), data_pearlite[100:]['c22'].values),
                                   'c33': np.append(np.append(data_spheroidite[100:]['c33'].values, data_network[100:]['c33'].values), data_pearlite[100:]['c33'].values),
                                   'c43': np.append(np.append(data_spheroidite[100:]['c43'].values, data_network[100:]['c43'].values), data_pearlite[100:]['c43'].values),
                                   'c53': np.append(np.append(data_spheroidite[100:]['c53'].values, data_network[100:]['c53'].values), data_pearlite[100:]['c53'].values),
                                   'type': np.append(np.append(np.ones(data_spheroidite[100:].shape[0]), 2*np.ones(data_network[100:].shape[0])), 3*np.ones(data_pearlite[100:].shape[0]))})


In [24]:
svm_1 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
ovo_1 = OneVsOneClassifier (svm_1, n_jobs=None)
score  = cross_validate (ovo_1, list(train_data['c12'].values), list(train_data['type'].values), cv=10)
print ("Multi classifier CV error for Layer-1 :", 1.0 - score['test_score'].mean())

svm_2 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
ovo_2 = OneVsOneClassifier (svm_2, n_jobs=None)
score  = cross_validate (ovo_2, list(train_data['c22'].values), list(train_data['type'].values), cv=10)
print ("Multi classifier CV error for Layer-2 :", 1.0 - score['test_score'].mean())

svm_3 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
ovo_3 = OneVsOneClassifier (svm_3, n_jobs=None)
score  = cross_validate (ovo_3, list(train_data['c33'].values), list(train_data['type'].values), cv=10)
print ("Multi classifier CV error for Layer-3 :", 1.0 - score['test_score'].mean())

svm_4 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
ovo_4 = OneVsOneClassifier (svm_4, n_jobs=None)
score  = cross_validate (ovo_4, list(train_data['c43'].values), list(train_data['type'].values), cv=10)
print ("Multi classifier CV error for Layer-4 :", 1.0 - score['test_score'].mean())

svm_5 = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
ovo_5 = OneVsOneClassifier (svm_5, n_jobs=None)
score  = cross_validate (ovo_5, list(train_data['c53'].values), list(train_data['type'].values), cv=10)
print ("Multi classifier CV error for Layer-5 :", 1.0 - score['test_score'].mean())

Multi classifier CV error for Layer-1 : 0.6633333333333333
Multi classifier CV error for Layer-2 : 0.6466666666666667
Multi classifier CV error for Layer-3 : 0.64
Multi classifier CV error for Layer-4 : 0.6499999999999999
Multi classifier CV error for Layer-5 : 0.050000000000000044


In [ ]:
# Multi classifier CV error for Layer-1 : 0.6633333333333333
# Multi classifier CV error for Layer-2 : 0.6466666666666667
# Multi classifier CV error for Layer-3 : 0.64
# Multi classifier CV error for Layer-4 : 0.6499999999999999
# Multi classifier CV error for Layer-5 : 0.050000000000000044

In [25]:
svm_ = svm.SVC(kernel='rbf', gamma='auto', random_state=0)
ovo = OneVsOneClassifier (svm_, n_jobs=None)
ovo.fit(list(train_data['c53'].values), list(train_data['type'].values))
score = ovo.score(list(test_data['c53']), list(test_data['type']))
print ("Multi classifier test error for Layer-5 :", 1.0 - score)

Multi classifier test error for Layer-5 : 0.03170731707317076


In [ ]:
# Multi classifier test error for Layer-5 : 0.03170731707317076

In [26]:
test_data = pd.DataFrame ({'c53': np.append(np.append(data_spheroidite[100:]['c53'].values, 
                            data_network[100:]['c53'].values),data_pearlite[100:]['c53']),
                           'type': np.append(np.append(np.ones(data_spheroidite[100:].shape[0]), 
                            2*np.ones(data_network[100:].shape[0])), 3*np.ones(data_pearlite[100:].shape[0]))})


In [27]:
pred_1 = sp_nt_clf53.predict(list(test_data['c53'].values))
pred_2 = nt_pl_clf53.predict(list(test_data['c53'].values))
pred_3 = pl_sp_clf53.predict(list(test_data['c53'].values))
pred = mode(np.concatenate((pred_1.reshape((pred_1.shape[0], 1)), pred_2.reshape((pred_2.shape[0], 1)),
                pred_3.reshape((pred_3.shape[0], 1)), np.zeros((pred_3.shape[0],1))), axis = 1), axis=1)[0]
truth_val = test_data['type'].values
truth_val = truth_val.reshape((truth_val.shape[0], 1))
print ("Total test error in 1 vs 1 : ", 1.0 - (truth_val[truth_val == pred].shape[0]/truth_val.shape[0]))

Total test error in 1 vs 1 :  0.03170731707317076


In [28]:
# Total test error in 1 vs 1 :  0.03170731707317076

In [29]:
# For spherodite
pred_1 = sp_nt_clf53.predict(list(data_spheroidite[100:]['c53'].values))
pred_2 = pl_sp_clf53.predict(list(data_spheroidite[100:]['c53'].values))
correct_pred_sph = np.array(pred_1 == 1.0) & np.array(pred_2 == 1.0)

# For network
pred_1 = nt_pl_clf53.predict(list(data_network[100:]['c53'].values))
pred_2 = sp_nt_clf53.predict(list(data_network[100:]['c53'].values))
correct_pred_ntwk = np.array(pred_1 == 2.0) & np.array(pred_2 == 2.0)

# For pearlite
pred_1 = pl_sp_clf53.predict(list(data_pearlite[100:]['c53'].values))
pred_2 = nt_pl_clf53.predict(list(data_pearlite[100:]['c53'].values))
correct_pred_pl = np.array(pred_1 == 3.0) & np.array(pred_2 == 3.0)

In [30]:
prediction = np.append(np.append(correct_pred_sph, correct_pred_ntwk), correct_pred_pl)

total_performance = prediction[prediction == True].shape[0]/prediction.shape[0]
print ("Total error : ", 1.0 - total_performance)

Total error :  0.03170731707317076


In [31]:
# Total test error :  0.03170731707317076

In [32]:
data_spheroidite.to_csv('data/sperodite.csv', sep=',')
data_network.to_csv('data/network.csv', sep=',')
data_pearlite.to_csv('data/pearlite.csv', sep=',')

In [33]:
data_spheroidite.to_csv('data/sperodite_t.csv', sep='\t')
data_network.to_csv('data/network_t.csv', sep='\t')
data_pearlite.to_csv('data/pearlite_t.csv', sep='\t')

In [48]:
img = image.img_to_array(image.load_img('data/micrograph/micrograph1.tif'))
img = img[0:484,:,:]
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
c12, c22, c33, c43, c53 = intermediate_layer_model.predict(img)

In [49]:
c12.shape

(1, 242, 322, 64)

In [50]:
c22.shape

(1, 121, 161, 128)

In [51]:
c33.shape

(1, 60, 80, 256)

In [53]:
c43.shape

(1, 30, 40, 512)

In [54]:
c53.shape

(1, 15, 20, 512)

In [58]:
print (model.get_layer(index=3) )
print (model.get_layer(index=6))
print (model.get_layer(index=10))
print (model.get_layer(index=14))
print (model.get_layer(index=18))

In [62]:
model.layers